In [52]:
import os
import json
import random
import pandas as pd
import numpy as np
from tqdm import tqdm

In [15]:
import wave

In [2]:
# MOS Score
test_mos = pd.read_csv(
    "/data/share/data/Speech/somos/training_files/split1/clean/test_mos_list.csv")
test_mos

,utteranceId,mean
0,LJ002-0181_110.wav,4.000000
1,LJ002-0181_148.wav,4.000000
2,LJ003-0083_177.wav,3.727273
3,LJ003-0173_007.wav,3.400000
4,LJ003-0173_166.wav,3.000000
...,...,...
2995,wiki_0096_105.wav,3.714286
2996,wiki_0097_061.wav,3.100000
2997,wiki_0098_115.wav,2.000000
2998,wiki_0098_124.wav,2.333333


In [16]:
# transcript 
lines = []
# path = "/data/share/data/Speech/somos/transcript/additional_sentences.txt"
# with open(path) as f:
#     for line in f:
#         lines.append(line)
path = "/data/share/data/Speech/somos/transcript/all_sentences.txt"
with open(path) as f:
    for line in f:
        lines.append(line)
transcripts = {}
for line in lines:
    items = line.split('\t')
    assert len(items) == 2
    transcripts[items[0]] = items[1].strip()

In [17]:
len(transcripts)

2000

In [18]:
# data 
# /data/share/data/Speech/somos/audios/{utteranceId}
data = []
for i in range(len(test_mos)):
    uttId, mos = test_mos['utteranceId'][i], test_mos['mean'][i]
    uttId0 = uttId.rsplit('_', 1)[0]
    if 'gt_' in uttId0:
        uttId0 = uttId0.replace('gt_', '')
    path = f"/data/share/data/Speech/somos/audios/{uttId}"
    assert os.path.exists(path)
    assert uttId0 in transcripts
    data.append({
        'uttId': uttId.replace('.wav', ''),
        'mos': mos,
        'text': transcripts[uttId0],
        'path': path
    })

In [19]:
uttId0, uttId

('wiki_0099', 'wiki_0099_056.wav')

In [20]:
len(test_mos), len(data)

(3000, 3000)

In [21]:
data[0]

{'uttId': 'LJ002-0181_110',
 'mos': 4.0,
 'text': "to those in other London prisons, for Newgate was not the only place of durance for these unfortunate people. There were also the King's Bench.",
 'path': '/data/share/data/Speech/somos/audios/LJ002-0181_110.wav'}

In [22]:
len(data)

3000

# Verbosity
- 1 vs 2
- 2 vs 3
- 3 vs 4

In [29]:
def repeat_wav(input_path, output_path, repetitions):
    """
    Create a new WAV file by repeating the audio content of the original WAV file N times.

    Parameters:
        input_path (str): Path to the input WAV file.
        output_path (str): Path to save the new WAV file.
        repetitions (int): Number of times to repeat the audio.
    """
    # Open the input WAV file
    with wave.open(input_path, 'rb') as wav_in:
        params = wav_in.getparams()  # Get audio parameters
        n_channels = wav_in.getnchannels()
        sample_width = wav_in.getsampwidth()
        frame_rate = wav_in.getframerate()
        n_frames = wav_in.getnframes()

        # Read the audio data
        audio_data = wav_in.readframes(n_frames)

    # Convert audio data to a numpy array
    dtype = np.int16 if sample_width == 2 else np.uint8  # Handle 16-bit or 8-bit audio
    audio_array = np.frombuffer(audio_data, dtype=dtype)

    # Repeat the audio array N times
    repeated_audio = np.tile(audio_array, repetitions)

    # Convert back to bytes
    repeated_audio_bytes = repeated_audio.tobytes()

    # Write the repeated audio to the output WAV file
    with wave.open(output_path, 'wb') as wav_out:
        wav_out.setnchannels(n_channels)
        wav_out.setsampwidth(sample_width)
        wav_out.setframerate(frame_rate)
        wav_out.writeframes(repeated_audio_bytes)


In [30]:
random.shuffle(data)

In [31]:
data80 = data[:80]

In [32]:
len(data80)

80

In [48]:
x

{'uttId': 'news_2009_0047_196',
 'mos': 3.363636363636364,
 'text': 'There is never any trouble with Ajax and Celtic fans.',
 'path': '/data/share/data/Speech/somos/audios/news_2009_0047_196.wav'}

In [60]:
# 1 vs 2 verbosity
verbosity_data = []
for x in tqdm(data80):
    N = 2
    path1 = x['path']
    path2 = f"/data/workspace/ppotsawee/audioLM-as-judge/data/verbosity/repeated_somos/N{N}_{path1.split('/')[-1]}"
    repeat_wav(path1, path2, repetitions=N)
    item1 = {
        'repetitions': 1,
        'path': path1,
        'text': x['text'],
        'mos': x['mos'],
        'uttId': x['uttId']
    }
    item2 = {
        'repetitions': 2,
        'path': path2,
        'text': x['text'],
        'mos': x['mos'],
        'uttId': x['uttId']
    }
    items = [item1, item2]
    random.shuffle(items)
    verbosity_data.append(items)
with open("/data/workspace/ppotsawee/audioLM-as-judge/data/data_somos_verbosity_n1n2.json", "w") as f:
    json.dump(verbosity_data, f, indent=4)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 896.75it/s]


In [63]:
# 2 vs 3 verbosity
verbosity_data = []
for x in tqdm(data80):
    path1 = x['path']
    path2 = f"/data/workspace/ppotsawee/audioLM-as-judge/data/verbosity/repeated_somos/N2_{path1.split('/')[-1]}"
    path3 = f"/data/workspace/ppotsawee/audioLM-as-judge/data/verbosity/repeated_somos/N3_{path1.split('/')[-1]}"
    repeat_wav(path1, path2, repetitions=2)
    repeat_wav(path1, path3, repetitions=3)
    item3 = {
        'repetitions': 3,
        'path': path3,
        'text': x['text'],
        'mos': x['mos'],
        'uttId': x['uttId']
    }
    item2 = {
        'repetitions': 2,
        'path': path2,
        'text': x['text'],
        'mos': x['mos'],
        'uttId': x['uttId']
    }
    items = [item3, item2]
    random.shuffle(items)
    verbosity_data.append(items)
with open("/data/workspace/ppotsawee/audioLM-as-judge/data/data_somos_verbosity_n2n3.json", "w") as f:
    json.dump(verbosity_data, f, indent=4)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 415.64it/s]


In [67]:
# 3 vs 4 verbosity
verbosity_data = []
for x in tqdm(data80):
    path1 = x['path']
    path4 = f"/data/workspace/ppotsawee/audioLM-as-judge/data/verbosity/repeated_somos/N4_{path1.split('/')[-1]}"
    path3 = f"/data/workspace/ppotsawee/audioLM-as-judge/data/verbosity/repeated_somos/N3_{path1.split('/')[-1]}"
    repeat_wav(path1, path4, repetitions=4)
    repeat_wav(path1, path3, repetitions=3)
    item3 = {
        'repetitions': 3,
        'path': path3,
        'text': x['text'],
        'mos': x['mos'],
        'uttId': x['uttId']
    }
    item4 = {
        'repetitions': 4,
        'path': path2,
        'text': x['text'],
        'mos': x['mos'],
        'uttId': x['uttId']
    }
    items = [item3, item4]
    random.shuffle(items)
    verbosity_data.append(items)
with open("/data/workspace/ppotsawee/audioLM-as-judge/data/data_somos_verbosity_n3n4.json", "w") as f:
    json.dump(verbosity_data, f, indent=4)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 469.68it/s]


# Analysis

In [73]:
import json
import re

In [86]:
def read_output_file(path):
    # path = "../experiments/somos/ab_testing/diff15_prompt2_verbosity_n1n2.txt"
    outputs = []
    with open(path, "r") as f:
        for line in f:
            x = json.loads(line)
            outputs.append(x)
    
    count_shorter, count_longer, count_tie, count_all = 0, 0, 0, 0
    for output in outputs:
        audio_a, audio_b = output['data']
        judge_text = output['response']
    
        # typo lol
        # judge_text = judge_text.replace("</verduct>", "</verdict>")
        # judge_text = judge_text.replace("</Verdict>", "</verdict>")
        # judge_text = judge_text.replace("</verddict>", "</verdict>")
        # judge_text = judge_text.replace("<Verdict>", "<verdict>")
        # judge_text = judge_text.replace("<verdict>B</verdict>", "<verdict>[[B]]</verdict>")
        # judge_text = judge_text.replace("Verdict: [[A]] </verdict>", "<verdict>[[A]]</verdict>")
        
        # Extract content between <verdict> and </verdict>
        verdict_match = re.search(r'<verdict>(.*?)</verdict>', judge_text, re.DOTALL)
        verdict_text = verdict_match.group(1).strip() if verdict_match else None
        assert verdict_text in ["[[A]]", "[[B]]", "[[Tie]]"], f"Unexpected verdict: {judge_text}"
    
        if verdict_text == "[[A]]":
            if audio_a['repetitions'] > audio_b['repetitions']:
                count_longer += 1
            else:
                count_shorter += 1
        elif verdict_text == "[[B]]":
            if audio_a['repetitions'] < audio_b['repetitions']:
                count_longer += 1
            else:
                count_shorter += 1
        else:
            count_tie += 1
        count_all += 1
    print("total:", count_all)
    print("-----------------------")
    print("Shorter: {:.2f}%".format(count_shorter/count_all*100))
    print("Tie:     {:.2f}%".format(count_tie/count_all*100))
    print("Longer:  {:.2f}%".format(count_longer/count_all*100))
    # return predictions

In [87]:
read_output_file("../experiments/somos/ab_testing/diff15_prompt2_verbosity_n1n2.txt")

total: 80
-----------------------
Shorter: 60.00%
Tie:     2.50%
Longer:  37.50%


In [88]:
read_output_file("../experiments/somos/ab_testing/diff15_prompt2_verbosity_n2n3.txt")

total: 80
-----------------------
Shorter: 62.50%
Tie:     0.00%
Longer:  37.50%


In [89]:
read_output_file("../experiments/somos/ab_testing/diff15_prompt2_verbosity_n3n4.txt")

total: 80
-----------------------
Shorter: 73.75%
Tie:     1.25%
Longer:  25.00%
